In [1]:
from postgres_connect import *
import requests
import logging

/Users/apple/Documents/akshaygrows/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Getting Data

In [2]:
query = '''
select r.rider_id, node_node_id, rider_pip.rider_name, rider_pip.hub, rider_pip.shipping_city as city, r.locus_rider_id
from rider_pip 
left join application_db.rider as r on r.locus_rider_id = rider_pip.rider_id
where 
        pip_status = 'closed' 
and     pip_result = 'failed'
and     pip_date = '2024-01-15'
order by node_node_id
'''

In [3]:
path = 'tunnel-ssh.cer'
# printthis('yes')
conn = get_conn('Yes',path)
df_block = pd.read_sql(query,conn)
conn.close()

Tunnel Started
Connection Made


In [4]:
df_block

,rider_id,node_node_id,rider_name,hub,city,locus_rider_id
0,3827,2,lakshmi pathi,DS BLR BOMM,Bangalore,lakshmi_9972726538_02
1,3919,2,karthik Gowda,DS BLR BOMM,Bangalore,karthik_9108468511_02
2,1938,2,Raju B,DS BLR BOMM,Bangalore,Raju_B_2
3,3508,3,arshad Shaikh,DS BOM KRL,Mumbai,arshad_Shaikh__3_1701335934
4,3544,3,REHAN KHAN,DS BOM KRL,Mumbai,REHAN_KHAN_3_1701500715
5,3758,3,Saif Shaikh,DS BOM KRL,Mumbai,Saif_8767337118_03
6,3908,3,Huzaifa Raeen,DS BOM KRL,Mumbai,Huzaifa_8591894061_03
7,3680,4,Suresh Manthurthi,DS HYD KMP,Hyderabad,_9701430190_04
8,2405,4,T Vikram Singh,DS HYD KMP,Hyderabad,T_Vikram_Singh__4
9,3711,4,Bala Krishna,DS HYD KMP,Hyderabad,Bala_8185059013_04


Initiating Blocking

In [11]:
# Configure logging
logging.basicConfig(level=logging.INFO)

# Constants
BASE_URL = 'http://grow-simplee-nlb-prod-a264c46571856f67.elb.ap-south-1.amazonaws.com:9001'
API_ENDPOINT = '/rider/delist'
HEADERS_BASE = {
    'name': 'void',  # Change this value if needed
    'Content-Type': 'application/json',
}

# Create a session for making multiple requests
session = requests.Session()

# List to store rider IDs with timeout
timeout_rider_ids = []

for index, row in df_block.iterrows():
    headers = HEADERS_BASE.copy()
    headers['node_id'] = str(row['node_node_id'])

    data = {
        'delistType': 'BLACKLIST',
        'riderIds': [str(row['rider_id'])],
    }

    url = f"{BASE_URL}{API_ENDPOINT}"

    try:
        response = session.put(url, headers=headers, json=data, timeout=5)
        response.raise_for_status()  # Raise an HTTPError for bad responses

        json_response = response.json()
        message = json_response.get('message', '')
        logging.info(f"Request for rider_id {row['rider_id']} - {message}")

        data = json_response.get('data', [])
        for item in data:
            rider_id = item.get('riderId', '')
            rider_name = item.get('riderName', '')
            error_message = item.get('message', '')
            logging.info(f"    Rider ID: {rider_id}, Rider Name: {rider_name}, Error Message: {error_message}")

    except requests.exceptions.Timeout:
        logging.error(f"Timeout occurred for rider_id {row['rider_id']}")
        timeout_rider_ids.append(row['rider_id'])

    except requests.exceptions.HTTPError as errh:
        logging.error(f"HTTP Error: {errh}")
    except requests.exceptions.RequestException as err:
        logging.error(f"Request Exception: {err}")

print("Rider blocking completed updating database")

# Print or use timeout_rider_ids as needed
print("Rider IDs with timeout:", timeout_rider_ids)


INFO:root:Request for rider_id 3827 - Success
INFO:root:Request for rider_id 1938 - Success
INFO:root:Request for rider_id 3919 - Success
INFO:root:Request for rider_id 3758 - Success
INFO:root:Request for rider_id 3508 - Success
INFO:root:Request for rider_id 3908 - Success
INFO:root:Request for rider_id 3544 - Success
INFO:root:Request for rider_id 3711 - Success
INFO:root:Request for rider_id 2405 - Success
INFO:root:Request for rider_id 3680 - Success
INFO:root:Request for rider_id 3454 - Success
INFO:root:Request for rider_id 1503 - Partial Success
INFO:root:    Rider ID: 1503, Rider Name: Sandeep VLS, Error Message: Some error occurred
INFO:root:Request for rider_id 3522 - Success
INFO:root:Request for rider_id 3487 - Success
INFO:root:Request for rider_id 1520 - Success
INFO:root:Request for rider_id 3846 - Success
INFO:root:Request for rider_id 3960 - Success
INFO:root:Request for rider_id 3883 - Success
INFO:root:Request for rider_id 3574 - Success
INFO:root:Request for rider_

Rider blocking completed updating database
Rider IDs with timeout: []


Updating Tables

In [4]:
conn = get_conn('Yes',path)
cursor = conn.cursor()
update_query1 = '''
    UPDATE rider_pip
    SET pip_status = 'final'
    WHERE pip_status = 'closed';
'''
update_query2 = '''
delete from rider_pip where pip_status = 'running' and rider_id in 
(select rider_id from rider_pip where pip_status = 'final' and pip_result = 'failed')
'''
cursor.execute(update_query1)
print("Updated all closed status to final")
cursor.execute(update_query2)
print("Deleted blocked riders from running pip")
conn.commit()
cursor.close()
conn.close()

Tunnel Started
Connection Made


Sending Closure Mails

In [5]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime

subject = f"Rider PIP Blocked Riders Report - {datetime.today().strftime('%d/%m/%Y')}"
from_email = "akshay@blitznow.in"  # Replace with your email
to_email = "akshay@blitznow.in, sanjay@blitznow.in, david.kumar@blitznow.in, ajay.verma@blitznow.in, biswojit.jena@blitznow.in, chandrakanth.sunkoji@blitznow.in, rahul.yadav@blitznow.in, rahul.lochav@blitznow.in, sourabh.vijay@blitznow.in, sourabh.vijayvergia@blitznow.in, syed.afrid@blitznow.in, anandbellary.hindushree@blitznow.in, dilip.kumar@blitznow.in, ma.saifullah@blitznow.in, shankar.poddar@blitznow.in, sumit.kamble@blitznow.in, anil@blitznow.in"


# Group the DataFrame by 'city' and 'hub' and format the data
blocked_riders_by_city_hub = df_block.groupby(['city', 'hub'])
email_content = "<p>Hi Team, These are the riders blocked tonight</p>"

for (city, hub), group in blocked_riders_by_city_hub:
    email_content += f"<p><b>City:</b> {city} <b>Hub:</b> {hub}</p>"
    email_content += group[['locus_rider_id', 'rider_name']].to_html(index=False, justify='center', classes='table', escape=False)

# Add inline styles for the table header
email_content = email_content.replace('<th>', '<th style="background-color: #c7dceb;">')

# Compose the email
msg = MIMEMultipart()
msg['From'] = from_email
msg['To'] = to_email
msg['Subject'] = subject

# Attach the email content
msg.attach(MIMEText(email_content, 'html'))

# Connect to the SMTP server and send the email
try:
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(from_email, 'Outofblitz1!')  # Replace with your email password
        server.sendmail(from_email, to_email, msg.as_string())
    print("Email sent successfully.")
except Exception as e:
    print(f"Error sending email: {e}")


Email sent successfully.
